In [1]:
from gevent import monkey
monkey.patch_all(thread=False)
from gevent.queue import Queue
import gevent

import sys
sys.path.append('../../../00.module')
import UA
import Proxy

# = = = = = = = = = = = = = = =

import pandas as pd

output_correct = pd.DataFrame()

output_error = pd.DataFrame()

# = = = = = = = = = = = = = = =

input_ = pd.read_excel('./2.year.xlsx',
                       header=0,
                       dtype=str).fillna('')

length = len(input_)

print('总数量：' + str(length))
print()

# = = = = = = = = = = = = = = =

import requests
import json

from bs4 import BeautifulSoup

from lxml import etree

# = = = = = = = = = = = = = = =

import time
from datetime import datetime

# = = = = = = = = = = = = = = =

work = Queue()
for i in range(length):
    work.put_nowait(i)

def crawler():
    global output_correct, output_error
    
    while not work.empty():
        i = work.get_nowait()
        
        # = = = = = = = = = = = = = = =
        
        crawler_status = 'error'
        for _ in range(101):
            try:
                jsn = '{"jsn":{"tab":"catalog","make":"' + input_.loc[i, 'Make'] + '","year":"' + input_.loc[i, 'Year'] + '","nodetype":"year"}}'

                payload = {'func': 'navnode_fetch',
                           'payload': jsn,
                           'api_json_request': '1'}

                resp = requests.post('https://www.rockauto.com/catalog/catalogapi.php',
                                     data=payload,
                                     headers=UA.get_User_Agent_Requests(),
                                     proxies=Proxy.get_Proxy_Requests()).text

                resp = json.loads(resp)
                
                # = = = = = = = = = = = = = = =
                
                html = resp['html_fill_sections']['navchildren[]']
                
                soup = BeautifulSoup(html, 'lxml')
                
                # = = = = = = = = = = = = = = =
                
                html = etree.HTML(str(soup))
                
                security_code = html.xpath('//div[@class="center"]/div')

                # = = = = = = = = = = = = = = =
                
                if len(security_code) != 0 and 'security code' in security_code[0].text:
                    continue
                    
                # = = = = = = = = = = = = = = =

                list_model = [model.strip() for model in html.xpath('//td[@class="nlabel"]/a/text()')]

                # = = = = = = = = = = = = = = =
                
                list_url = ['https://www.rockauto.com'+url.strip() for url in html.xpath('//td[@class="niconspace ncollapsedicon"]/a/@href')]
                
                # = = = = = = = = = = = = = = =
                
                df_temp = pd.DataFrame({'Tab_Label': input_.loc[i, 'Tab_Label'],
                                        'No.': int(input_.loc[i, 'No.']),
                                        'Make': input_.loc[i, 'Make'],
                                        'Year': input_.loc[i, 'Year'],
                                        'Model': list_model,
                                        'Url': list_url})

                output_correct = pd.concat([output_correct, df_temp], ignore_index=True).fillna('')
                
                # = = = = = = = = = = = = = = =
                
                crawler_status = 'ok'
                
                break
                
            except:
                time.sleep(0.3)
                continue

        if crawler_status != 'ok':
            df_temp = pd.DataFrame([{'Tab_Label': input_.loc[i, 'Tab_Label'],
                                     'No.': int(input_.loc[i, 'No.']),
                                     'Make': input_.loc[i, 'Make'],
                                     'Year': input_.loc[i, 'Year'],
                                     'Url': input_.loc[i, 'Url']}])
    
            output_error = pd.concat([output_error, df_temp], ignore_index=True).fillna('')

        print(input_.loc[i, 'Make'] + ' - ' + input_.loc[i, 'Year'] +  '  <->  [' + crawler_status + '] - 剩余数量：' + str(work.qsize()) + ' - 当前时间：' + datetime.now().strftime('%H:%M:%S'))

# = = = = = = = = = = = = = = =

list_task = []
for _ in range(20):
    task = gevent.spawn(crawler)
    list_task.append(task)
gevent.joinall(list_task)

print()
print('爬虫输出中')
output_correct = output_correct.drop_duplicates(ignore_index=True)
output_correct = output_correct.sort_values(by=['No.', 'Year', 'Model'], ascending=[True, False, True])
output_correct.to_excel('./3.model-' + datetime.now().strftime('%Y%m%d_%H%M%S') + '.xlsx', index=False)
if len(output_error) != 0:
    output_error = output_error.drop_duplicates(ignore_index=True)
    output_error = output_error.sort_values(by=['No.', 'Year'], ascending=[True, False])
    output_error.to_excel('./model_error.xlsx', index=False)
    print()
    print('爬虫存在error')
print()
print('爬虫结束')

总数量：6191

ABARTH - 1966  <->  [ok] - 剩余数量：6171 - 当前时间：15:03:25
ABARTH - 1952  <->  [ok] - 剩余数量：6170 - 当前时间：15:03:25
ABARTH - 1950  <->  [ok] - 剩余数量：6169 - 当前时间：15:03:25
ABARTH - 1961  <->  [ok] - 剩余数量：6168 - 当前时间：15:03:25
ABARTH - 1968  <->  [ok] - 剩余数量：6167 - 当前时间：15:03:25
ABARTH - 1951  <->  [ok] - 剩余数量：6166 - 当前时间：15:03:25
ABARTH - 1958  <->  [ok] - 剩余数量：6165 - 当前时间：15:03:25
ABARTH - 1955  <->  [ok] - 剩余数量：6164 - 当前时间：15:03:26
ABARTH - 1963  <->  [ok] - 剩余数量：6163 - 当前时间：15:03:26
ABARTH - 1953  <->  [ok] - 剩余数量：6162 - 当前时间：15:03:26
ABARTH - 1960  <->  [ok] - 剩余数量：6161 - 当前时间：15:03:26
ABARTH - 1957  <->  [ok] - 剩余数量：6160 - 当前时间：15:03:26
ABARTH - 1954  <->  [ok] - 剩余数量：6159 - 当前时间：15:03:26
ABARTH - 1959  <->  [ok] - 剩余数量：6158 - 当前时间：15:03:26
ABARTH - 1969  <->  [ok] - 剩余数量：6157 - 当前时间：15:03:26
AC - 1973  <->  [ok] - 剩余数量：6156 - 当前时间：15:03:26
ABARTH - 1965  <->  [ok] - 剩余数量：6155 - 当前时间：15:03:27
AC - 1972  <->  [ok] - 剩余数量：6154 - 当前时间：15:03:27
AC - 1962  <->  [ok] - 剩余数量：6153 - 当前时间：15:0

ALLARD - 1950  <->  [ok] - 剩余数量：6017 - 当前时间：15:03:46
ALPINE - 1972  <->  [ok] - 剩余数量：6016 - 当前时间：15:03:47
ALPINE - 1971  <->  [ok] - 剩余数量：6015 - 当前时间：15:03:47
ALLARD - 1948  <->  [ok] - 剩余数量：6014 - 当前时间：15:03:47
ALPINE - 1969  <->  [ok] - 剩余数量：6013 - 当前时间：15:03:47
ALPINE - 1970  <->  [ok] - 剩余数量：6012 - 当前时间：15:03:47
ALLSTATE - 1952  <->  [ok] - 剩余数量：6011 - 当前时间：15:03:48
ALFA ROMEO - 1948  <->  [ok] - 剩余数量：6010 - 当前时间：15:03:48
ALPINE - 1964  <->  [ok] - 剩余数量：6009 - 当前时间：15:03:48
ALPINE - 1966  <->  [ok] - 剩余数量：6008 - 当前时间：15:03:48
ALPINE - 1965  <->  [ok] - 剩余数量：6007 - 当前时间：15:03:49
ALPINE - 1973  <->  [ok] - 剩余数量：6006 - 当前时间：15:03:49
ALVIS - 1963  <->  [ok] - 剩余数量：6005 - 当前时间：15:03:50
ALVIS - 1962  <->  [ok] - 剩余数量：6004 - 当前时间：15:03:50
ALVIS - 1964  <->  [ok] - 剩余数量：6003 - 当前时间：15:03:50
ACURA - 2005  <->  [ok] - 剩余数量：6002 - 当前时间：15:03:50
ALLARD - 1953  <->  [ok] - 剩余数量：6001 - 当前时间：15:03:50
ALVIS - 1967  <->  [ok] - 剩余数量：6000 - 当前时间：15:03:50
ALPINE - 1963  <->  [ok] - 剩余数量：5999 - 当前时间：1

ASTON MARTIN - 1999  <->  [ok] - 剩余数量：5876 - 当前时间：15:04:13
ASTON MARTIN - 2001  <->  [ok] - 剩余数量：5875 - 当前时间：15:04:13
ASTON MARTIN - 1992  <->  [ok] - 剩余数量：5874 - 当前时间：15:04:14
ASTON MARTIN - 1994  <->  [ok] - 剩余数量：5873 - 当前时间：15:04:14
ASTON MARTIN - 1991  <->  [ok] - 剩余数量：5872 - 当前时间：15:04:14
ASTON MARTIN - 1990  <->  [ok] - 剩余数量：5871 - 当前时间：15:04:14
APOLLO - 1965  <->  [ok] - 剩余数量：5870 - 当前时间：15:04:14
ASTON MARTIN - 1984  <->  [ok] - 剩余数量：5869 - 当前时间：15:04:15
ASTON MARTIN - 1987  <->  [ok] - 剩余数量：5868 - 当前时间：15:04:15
ASTON MARTIN - 1989  <->  [ok] - 剩余数量：5867 - 当前时间：15:04:15
ASTON MARTIN - 1986  <->  [ok] - 剩余数量：5866 - 当前时间：15:04:15
ASTON MARTIN - 1993  <->  [ok] - 剩余数量：5865 - 当前时间：15:04:15
ASTON MARTIN - 1981  <->  [ok] - 剩余数量：5864 - 当前时间：15:04:16
ASTON MARTIN - 1980  <->  [ok] - 剩余数量：5863 - 当前时间：15:04:16
ASTON MARTIN - 1978  <->  [ok] - 剩余数量：5862 - 当前时间：15:04:16
ARNOLT-BRISTOL - 1960  <->  [ok] - 剩余数量：5861 - 当前时间：15:04:16
ASTON MARTIN - 2005  <->  [ok] - 剩余数量：5860 - 当前时间：15:04:17
A

AUSTIN - 1947  <->  [ok] - 剩余数量：5725 - 当前时间：15:04:46
AUSTIN - 1956  <->  [ok] - 剩余数量：5724 - 当前时间：15:04:46
AUSTIN - 1941  <->  [ok] - 剩余数量：5723 - 当前时间：15:04:46
AUSTIN - 1937  <->  [ok] - 剩余数量：5722 - 当前时间：15:04:47
AUSTIN - 1938  <->  [ok] - 剩余数量：5721 - 当前时间：15:04:47
AUSTIN-HEALEY - 1969  <->  [ok] - 剩余数量：5720 - 当前时间：15:04:47
ASTON MARTIN - 2004  <->  [ok] - 剩余数量：5719 - 当前时间：15:04:47
AUSTIN - 1950  <->  [ok] - 剩余数量：5718 - 当前时间：15:04:47
AUSTIN - 1948  <->  [ok] - 剩余数量：5717 - 当前时间：15:04:48
AUSTIN-HEALEY - 1968  <->  [ok] - 剩余数量：5716 - 当前时间：15:04:48
AUDI - 1973  <->  [ok] - 剩余数量：5715 - 当前时间：15:04:48
AUSTIN-HEALEY - 1965  <->  [ok] - 剩余数量：5714 - 当前时间：15:04:48
AUSTIN - 1971  <->  [ok] - 剩余数量：5713 - 当前时间：15:04:48
AUSTIN-HEALEY - 1964  <->  [ok] - 剩余数量：5712 - 当前时间：15:04:49
AUSTIN-HEALEY - 1967  <->  [ok] - 剩余数量：5711 - 当前时间：15:04:49
AUSTIN-HEALEY - 1963  <->  [ok] - 剩余数量：5710 - 当前时间：15:04:49
AUSTIN-HEALEY - 1961  <->  [ok] - 剩余数量：5709 - 当前时间：15:04:49
AUSTIN-HEALEY - 1960  <->  [ok] - 剩余数量：5708 - 

BIZZARRINI - 1968  <->  [ok] - 剩余数量：5573 - 当前时间：15:05:14
BIZZARRINI - 1969  <->  [ok] - 剩余数量：5572 - 当前时间：15:05:14
BENTLEY - 2003  <->  [ok] - 剩余数量：5571 - 当前时间：15:05:14
BIZZARRINI - 1967  <->  [ok] - 剩余数量：5570 - 当前时间：15:05:14
BENTLEY - 1952  <->  [ok] - 剩余数量：5569 - 当前时间：15:05:15
BERKELEY - 1957  <->  [ok] - 剩余数量：5568 - 当前时间：15:05:15
AUDI - 1974  <->  [ok] - 剩余数量：5567 - 当前时间：15:05:15
BMW - 2024  <->  [ok] - 剩余数量：5566 - 当前时间：15:05:15
BERKELEY - 1960  <->  [ok] - 剩余数量：5565 - 当前时间：15:05:15
BMW - 2023  <->  [ok] - 剩余数量：5564 - 当前时间：15:05:15
BIZZARRINI - 1966  <->  [ok] - 剩余数量：5563 - 当前时间：15:05:16
BIZZARRINI - 1965  <->  [ok] - 剩余数量：5562 - 当前时间：15:05:16
BMW - 2022  <->  [ok] - 剩余数量：5561 - 当前时间：15:05:16
BMW - 2021  <->  [ok] - 剩余数量：5560 - 当前时间：15:05:16
BMW - 2017  <->  [ok] - 剩余数量：5559 - 当前时间：15:05:16
BMW - 2020  <->  [ok] - 剩余数量：5558 - 当前时间：15:05:17
BMW - 2018  <->  [ok] - 剩余数量：5557 - 当前时间：15:05:17
BMW - 2016  <->  [ok] - 剩余数量：5556 - 当前时间：15:05:17
BMW - 2019  <->  [ok] - 剩余数量：5555 - 当前时间：15:05

...

STUDEBAKER - 1928  <->  [ok] - 剩余数量：643 - 当前时间：15:21:28
STUTZ - 1971  <->  [ok] - 剩余数量：642 - 当前时间：15:21:28
STUDEBAKER - 1933  <->  [ok] - 剩余数量：641 - 当前时间：15:21:28
STUTZ - 1973  <->  [ok] - 剩余数量：640 - 当前时间：15:21:28
STUDEBAKER - 1936  <->  [ok] - 剩余数量：639 - 当前时间：15:21:28
STUTZ - 1970  <->  [ok] - 剩余数量：638 - 当前时间：15:21:29
STUDEBAKER - 1942  <->  [ok] - 剩余数量：637 - 当前时间：15:21:29
STUTZ - 1934  <->  [ok] - 剩余数量：636 - 当前时间：15:21:29
STUTZ - 1929  <->  [ok] - 剩余数量：635 - 当前时间：15:21:29
STUTZ - 1932  <->  [ok] - 剩余数量：634 - 当前时间：15:21:29
STUTZ - 1931  <->  [ok] - 剩余数量：633 - 当前时间：15:21:30
STUDEBAKER - 1927  <->  [ok] - 剩余数量：632 - 当前时间：15:21:30
STUTZ - 1972  <->  [ok] - 剩余数量：631 - 当前时间：15:21:30
STUTZ - 1928  <->  [ok] - 剩余数量：630 - 当前时间：15:21:30
STUDEBAKER - 1929  <->  [ok] - 剩余数量：629 - 当前时间：15:21:30
STUTZ - 1923  <->  [ok] - 剩余数量：628 - 当前时间：15:21:30
SUBARU - 2024  <->  [ok] - 剩余数量：627 - 当前时间：15:21:30
SUBARU - 2021  <->  [ok] - 剩余数量：626 - 当前时间：15:21:31
SUBARU - 2023  <->  [ok] - 剩余数量：625 - 当前时间：15:21:3

TATRA - 1964  <->  [ok] - 剩余数量：486 - 当前时间：15:22:17
TATRA - 1957  <->  [ok] - 剩余数量：485 - 当前时间：15:22:17
SUBARU - 2018  <->  [ok] - 剩余数量：484 - 当前时间：15:22:17
SUZUKI - 1989  <->  [ok] - 剩余数量：483 - 当前时间：15:22:18
SUZUKI - 1985  <->  [ok] - 剩余数量：482 - 当前时间：15:22:18
TATRA - 1953  <->  [ok] - 剩余数量：481 - 当前时间：15:22:18
TATRA - 1954  <->  [ok] - 剩余数量：480 - 当前时间：15:22:18
TATRA - 1955  <->  [ok] - 剩余数量：479 - 当前时间：15:22:19
TATRA - 1952  <->  [ok] - 剩余数量：478 - 当前时间：15:22:19
TATRA - 1951  <->  [ok] - 剩余数量：477 - 当前时间：15:22:19
TATRA - 1950  <->  [ok] - 剩余数量：476 - 当前时间：15:22:19
TATRA - 1960  <->  [ok] - 剩余数量：475 - 当前时间：15:22:19
PONTIAC - 1974  <->  [ok] - 剩余数量：474 - 当前时间：15:22:20
TESLA - 2021  <->  [ok] - 剩余数量：473 - 当前时间：15:22:20
TESLA - 2023  <->  [ok] - 剩余数量：472 - 当前时间：15:22:20
SUZUKI - 2023  <->  [ok] - 剩余数量：471 - 当前时间：15:22:20
TESLA - 2017  <->  [ok] - 剩余数量：470 - 当前时间：15:22:21
TESLA - 2014  <->  [ok] - 剩余数量：469 - 当前时间：15:22:21
TESLA - 2013  <->  [ok] - 剩余数量：468 - 当前时间：15:22:22
TESLA - 2018  <->  [ok] -

TVR - 1961  <->  [ok] - 剩余数量：325 - 当前时间：15:22:56
TVR - 1958  <->  [ok] - 剩余数量：324 - 当前时间：15:22:56
UAZ - 2020  <->  [ok] - 剩余数量：323 - 当前时间：15:22:56
UD - 2013  <->  [ok] - 剩余数量：322 - 当前时间：15:22:56
UAZ - 2019  <->  [ok] - 剩余数量：321 - 当前时间：15:22:57
UD - 2012  <->  [ok] - 剩余数量：320 - 当前时间：15:22:57
UD - 2011  <->  [ok] - 剩余数量：319 - 当前时间：15:22:57
TVR - 1957  <->  [ok] - 剩余数量：318 - 当前时间：15:22:57
UD - 2007  <->  [ok] - 剩余数量：317 - 当前时间：15:22:57
UD - 2010  <->  [ok] - 剩余数量：316 - 当前时间：15:22:58
UD - 2003  <->  [ok] - 剩余数量：315 - 当前时间：15:22:58
UD - 2006  <->  [ok] - 剩余数量：314 - 当前时间：15:22:58
UD - 2005  <->  [ok] - 剩余数量：313 - 当前时间：15:22:59
UD - 2002  <->  [ok] - 剩余数量：312 - 当前时间：15:22:59
UD - 2009  <->  [ok] - 剩余数量：311 - 当前时间：15:22:59
UD - 2000  <->  [ok] - 剩余数量：310 - 当前时间：15:22:59
UD - 1998  <->  [ok] - 剩余数量：309 - 当前时间：15:22:59
UD - 2008  <->  [ok] - 剩余数量：308 - 当前时间：15:23:00
UD - 1997  <->  [ok] - 剩余数量：307 - 当前时间：15:23:00
UD - 1996  <->  [ok] - 剩余数量：306 - 当前时间：15:23:00
TVR - 1967  <->  [ok] - 剩余数量：305 - 

VOLVO - 2011  <->  [ok] - 剩余数量：170 - 当前时间：15:23:37
VOLVO - 2013  <->  [ok] - 剩余数量：169 - 当前时间：15:23:37
VOLVO - 2009  <->  [ok] - 剩余数量：168 - 当前时间：15:23:38
VOLVO - 2010  <->  [ok] - 剩余数量：167 - 当前时间：15:23:38
VOLVO - 2007  <->  [ok] - 剩余数量：166 - 当前时间：15:23:39
VOLVO - 2005  <->  [ok] - 剩余数量：165 - 当前时间：15:23:39
VOLVO - 2003  <->  [ok] - 剩余数量：164 - 当前时间：15:23:40
VOLVO - 2004  <->  [ok] - 剩余数量：163 - 当前时间：15:23:40
VOLVO - 2021  <->  [ok] - 剩余数量：162 - 当前时间：15:23:41
VOLKSWAGEN - 1950  <->  [ok] - 剩余数量：161 - 当前时间：15:23:41
VOLVO - 2000  <->  [ok] - 剩余数量：160 - 当前时间：15:23:42
VOLVO - 1999  <->  [ok] - 剩余数量：159 - 当前时间：15:23:42
VOLVO - 1997  <->  [ok] - 剩余数量：158 - 当前时间：15:23:43
VOLVO - 2001  <->  [ok] - 剩余数量：157 - 当前时间：15:23:43
VOLVO - 1996  <->  [ok] - 剩余数量：156 - 当前时间：15:23:43
VOLVO - 1994  <->  [ok] - 剩余数量：155 - 当前时间：15:23:44
VOLVO - 1995  <->  [ok] - 剩余数量：154 - 当前时间：15:23:44
VOLVO - 1993  <->  [ok] - 剩余数量：153 - 当前时间：15:23:44
VOLVO - 2014  <->  [ok] - 剩余数量：152 - 当前时间：15:23:45
VOLVO - 1992  <->  [ok] - 

YELLOW CAB - 1929  <->  [ok] - 剩余数量：11 - 当前时间：15:25:07
SUBARU - 2004  <->  [ok] - 剩余数量：10 - 当前时间：15:25:07
YELLOW CAB - 1928  <->  [ok] - 剩余数量：9 - 当前时间：15:25:07
YUGO - 1992  <->  [ok] - 剩余数量：8 - 当前时间：15:25:08
YUGO - 1990  <->  [ok] - 剩余数量：7 - 当前时间：15:25:08
WORKHORSE - 2001  <->  [ok] - 剩余数量：6 - 当前时间：15:25:08
YUGO - 1989  <->  [ok] - 剩余数量：5 - 当前时间：15:25:08
WARTBURG - 1957  <->  [ok] - 剩余数量：4 - 当前时间：15:25:08
YELLOW CAB - 1927  <->  [ok] - 剩余数量：3 - 当前时间：15:25:08
YUGO - 1987  <->  [ok] - 剩余数量：2 - 当前时间：15:25:09
YUGO - 1991  <->  [ok] - 剩余数量：1 - 当前时间：15:25:09
WORKHORSE - 2007  <->  [ok] - 剩余数量：0 - 当前时间：15:25:09
YUGO - 1986  <->  [ok] - 剩余数量：0 - 当前时间：15:25:09
ZACUA - 2018  <->  [ok] - 剩余数量：0 - 当前时间：15:25:10
ZACUA - 2019  <->  [ok] - 剩余数量：0 - 当前时间：15:25:10
ZUNDAPP - 1957  <->  [ok] - 剩余数量：0 - 当前时间：15:25:10
YUGO - 1988  <->  [ok] - 剩余数量：0 - 当前时间：15:25:10
ZUNDAPP - 1956  <->  [ok] - 剩余数量：0 - 当前时间：15:25:10
ZUNDAPP - 1958  <->  [ok] - 剩余数量：0 - 当前时间：15:25:11
WOLSELEY - 1955  <->  [ok] - 剩余数量：0 - 当前时